In [3]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [4]:
from model.rec_model import microgrid
from controller import rule_controller

In [5]:
setup = {
  "horizon": 25,
  "timestep": 1,
  "add_unbalanced_module": True,
  "loss_load_cost": 10,
  "overgeneration_cost": 2,
  "components": [
    {
      "id": "3c03f396-98f2-46c7-945c-3c95408b6355",
      "type": "LoadModule",
      "params": {
        "time_series_profile": "house"
      }
    },
    {
      "id": "32e548e7-8b69-49b6-b471-54908cd211d3",
      "type": "RenewableModule",
      "params": {
        "time_series_profile": "solar_1"
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d3",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 50,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d4",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 150,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "4ba2cfa4-7323-4f0a-ac7f-fa73f1177e49",
      "type": "GridModule",
      "params": {
        "max_import": 100,
        "max_export": 100,
        "cost_per_unit_co2": 0,
        "time_series_profile": "grid_1"
      }
    }
  ]
}

In [6]:
# Configurar la microgrid previamente
microgrid.setup(setup)

# Iniciar el controlador (se crea automáticamente la lista de prioridades)
rule_controller.setup()

In [7]:
microgrid.get_actions()

{'battery': [None, None], 'grid': [None]}

In [8]:
rule_controller.rbc.priority_list

[PriorityListElement(module=('battery', 0), module_actions=1, action=0, marginal_cost=0),
 PriorityListElement(module=('battery', 1), module_actions=1, action=0, marginal_cost=0),
 PriorityListElement(module=('grid', 0), module_actions=1, action=0, marginal_cost=0.2118)]

In [9]:
# Ejecutar un paso
rule_controller.step()

{'action': {'battery': [9.5, 9.5], 'grid': [9.329999999999998]},
 'observation': {'load': [0.6634929657524513],
  'battery': [array([0.3, 0.3]), array([0.43333333, 0.43333333])],
  'grid': [array([0.68652038, 0.82188295, 0.32648313, 0.        ])],
  'renewable': [0.0],
  'balancing': [array([], dtype=float64)]},
 'reward': -1.9760939999999996,
 'done': False,
 'info': {'load': [('absorbed_energy', 28.33)],
  'battery': [('provided_energy', 9.5), ('provided_energy', 9.5)],
  'grid': [('provided_energy', 9.329999999999998),
   ('co2_production', 2.4052739999999995)],
  'renewable': [('provided_energy', -0.0), ('curtailment', 0.0)],
  'balancing': [('provided_energy', 0.0)]}}

In [10]:
rule_controller.step()

{'action': {'battery': [9.5, 9.5], 'grid': [4.68]},
 'observation': {'load': [0.6397612619013784],
  'battery': [array([0.1, 0.1]), array([0.36666667, 0.36666667])],
  'grid': [array([0.69905956, 0.89694656, 0.25242342, 0.        ])],
  'renewable': [0.0],
  'balancing': [array([], dtype=float64)]},
 'reward': -1.0497239999999999,
 'done': False,
 'info': {'load': [('absorbed_energy', 23.68)],
  'battery': [('provided_energy', 9.5), ('provided_energy', 9.5)],
  'grid': [('provided_energy', 4.68), ('co2_production', 1.1919959999999998)],
  'renewable': [('provided_energy', -0.0), ('curtailment', 0.0)],
  'balancing': [('provided_energy', 0.0)]}}

In [11]:
microgrid.get_status()

{'load': [{'load_current': -25.35}],
 'renewable': [{'renewable_current': 0}],
 'balancing': [{}],
 'battery': [{'soc': 0.1, 'current_charge': 5.0},
  {'soc': 0.36666666666666664, 'current_charge': 55.0}],
 'grid': [{'import_price_current': 0.2259,
   'export_price_current': 0.1306,
   'co2_per_kwh_current': 0.2356,
   'grid_status_current': 1.0}],
 'total': [{'loads': -25.35, 'renewables': 0, 'batteries': 60.0}]}

In [12]:
microgrid.get_components()

[{'id': 'load_0', 'type': 'LoadModule'},
 {'id': 'renewable_0', 'type': 'RenewableModule'},
 {'id': 'balancing_0', 'type': 'UnbalancedEnergyModule'},
 {'id': 'battery_0', 'type': 'BatteryModule'},
 {'id': 'battery_1', 'type': 'BatteryModule'},
 {'id': 'grid_0', 'type': 'GridModule'}]

In [13]:
df = microgrid.get_log()

In [14]:
df

module_name     load                       renewable              \
module_number      0                               0               
field         reward load_met load_current    reward curtailment   
0                0.0    28.33       -28.33       0.0         0.0   
1                0.0    23.68       -23.68       0.0         0.0   

module_name                                    balancing            \
module_number                                          0             
field         renewable_used renewable_current    reward loss_load   
0                       -0.0                 0         0       0.0   
1                       -0.0                 0         0       0.0   

module_name                   ...              totals                       \
module_number                 ...                grid                        
field         overgeneration  ... grid_status_current import_price_current   
0                        0.0  ...                 1.0               0.2118   
1                        0.0  ...                 1.0               0.2243   

module_name                                                       \
module_number                   load                   renewable   
field            reward load_current load_met reward curtailment   
0             -1.976094       -28.33    28.33    0.0         0.0   
1             -1.049724       -23.68    23.68    0.0         0.0   

module_name                                            
module_number                                          
field         renewable_current renewable_used reward  
0                           0.0            0.0    0.0  
1                           0.0            0.0    0.0  

[2 rows x 71 columns]